In [ ]:
# different method for taking input inside the queue
# tensorboard to visualize your graph
# test on inception method

In [21]:
# stretching the cells
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# cuda settings
import os
import tensorflow as tf
os.environ["CUDA_VISIBLE_DEVICES"]="0"
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.50)

In [22]:
import tensorflow.contrib.slim as slim
import tensorflow.contrib.slim.nets
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [23]:
def read_data(data_path='train.tfrecords'):
    feature = {'image':tf.FixedLenFeature([],tf.string), 'label':tf.FixedLenFeature([],tf.int64)}
    filename_queue = tf.train.string_input_producer([data_path],num_epochs=10)

    reader = tf.TFRecordReader()
    _,serialzed_example = reader.read(filename_queue)

    features = tf.parse_single_example(serialzed_example,features=feature)

    image = tf.decode_raw(features["image"],tf.float32)
    label = tf.cast(features["label"],tf.int32)

    image = tf.reshape(image,[image_size,image_size,num_channels])
    label = tf.one_hot(label,depth=num_classes)
    images,labels = tf.train.shuffle_batch([image,label],batch_size=10,capacity=20,num_threads=1,min_after_dequeue=10)

    return (images,labels)

In [24]:
def layersGraph(model_name="vgg"):
    tf.reset_default_graph()
    temp= tf.Graph()
    if(model_name == "inception"):
        with temp.as_default():
            inception = slim.nets.inception
            image_size = inception.inception_v3.default_image_size
            x = tf.placeholder(tf.float32,(None,image_size,image_size,3))
            with slim.arg_scope(inception.inception_v3_arg_scope()):
                _,_ = inception.inception_v3(x,num_classes=10)
        
    elif(model_name == "vgg"):
        with temp.as_default():
            vgg = slim.nets.vgg
            image_size = vgg.vgg_16.default_image_size
            x = tf.placeholder(tf.float32,(None,image_size,image_size,3))
            with slim.arg_scope(vgg.vgg_arg_scope()):
                _,_ = vgg.vgg_16(x,num_classes=10)
            
    return temp

In [19]:
with layersGraph("inception").as_default():
    for layer in slim.get_variables_to_restore():
        print(layer)

<tf.Variable 'InceptionV3/Conv2d_1a_3x3/weights:0' shape=(3, 3, 3, 32) dtype=float32_ref>
<tf.Variable 'InceptionV3/Conv2d_1a_3x3/BatchNorm/beta:0' shape=(32,) dtype=float32_ref>
<tf.Variable 'InceptionV3/Conv2d_1a_3x3/BatchNorm/moving_mean:0' shape=(32,) dtype=float32_ref>
<tf.Variable 'InceptionV3/Conv2d_1a_3x3/BatchNorm/moving_variance:0' shape=(32,) dtype=float32_ref>
<tf.Variable 'InceptionV3/Conv2d_2a_3x3/weights:0' shape=(3, 3, 32, 32) dtype=float32_ref>
<tf.Variable 'InceptionV3/Conv2d_2a_3x3/BatchNorm/beta:0' shape=(32,) dtype=float32_ref>
<tf.Variable 'InceptionV3/Conv2d_2a_3x3/BatchNorm/moving_mean:0' shape=(32,) dtype=float32_ref>
<tf.Variable 'InceptionV3/Conv2d_2a_3x3/BatchNorm/moving_variance:0' shape=(32,) dtype=float32_ref>
<tf.Variable 'InceptionV3/Conv2d_2b_3x3/weights:0' shape=(3, 3, 32, 64) dtype=float32_ref>
<tf.Variable 'InceptionV3/Conv2d_2b_3x3/BatchNorm/beta:0' shape=(64,) dtype=float32_ref>
<tf.Variable 'InceptionV3/Conv2d_2b_3x3/BatchNorm/moving_mean:0' shap

<tf.Variable 'InceptionV3/Mixed_6e/Branch_2/Conv2d_0e_1x7/BatchNorm/moving_variance:0' shape=(192,) dtype=float32_ref>
<tf.Variable 'InceptionV3/Mixed_6e/Branch_3/Conv2d_0b_1x1/weights:0' shape=(1, 1, 768, 192) dtype=float32_ref>
<tf.Variable 'InceptionV3/Mixed_6e/Branch_3/Conv2d_0b_1x1/BatchNorm/beta:0' shape=(192,) dtype=float32_ref>
<tf.Variable 'InceptionV3/Mixed_6e/Branch_3/Conv2d_0b_1x1/BatchNorm/moving_mean:0' shape=(192,) dtype=float32_ref>
<tf.Variable 'InceptionV3/Mixed_6e/Branch_3/Conv2d_0b_1x1/BatchNorm/moving_variance:0' shape=(192,) dtype=float32_ref>
<tf.Variable 'InceptionV3/Mixed_7a/Branch_0/Conv2d_0a_1x1/weights:0' shape=(1, 1, 768, 192) dtype=float32_ref>
<tf.Variable 'InceptionV3/Mixed_7a/Branch_0/Conv2d_0a_1x1/BatchNorm/beta:0' shape=(192,) dtype=float32_ref>
<tf.Variable 'InceptionV3/Mixed_7a/Branch_0/Conv2d_0a_1x1/BatchNorm/moving_mean:0' shape=(192,) dtype=float32_ref>
<tf.Variable 'InceptionV3/Mixed_7a/Branch_0/Conv2d_0a_1x1/BatchNorm/moving_variance:0' shape=(

In [43]:
writer = tf.summary.FileWriter(logdir="./graph/")

In [44]:
image_size = slim.nets.inception.inception_v3.default_image_size
num_classes = 8
num_channels = 3
train_data_path = "train.tfrecords"
valid_data_path = "valid.tfrecords"

In [45]:
tf.reset_default_graph()
graph = tf.Graph()

print("Initializing the graph ..... !")

with graph.as_default():
    
    model_path = "./inception_v3.ckpt"
    assert(os.path.isfile(model_path))
    
    # Defining the placeholder variables
    x = tf.placeholder(tf.float32,shape=(None,image_size,image_size,num_channels))
    y = tf.placeholder(tf.int32,shape=(None,num_classes))
    
    inception = slim.nets.inception
    with slim.arg_scope(inception.inception_v3_arg_scope()):
        logits,endpoint = inception.inception_v3(x,num_classes,is_training=True)
    
    variables_to_restore = slim.get_variables_to_restore(exclude=['InceptionV3/Logits'])
    init_fn = slim.assign_from_checkpoint_fn(model_path,variables_to_restore)
    
    logit_variables = slim.get_variables('InceptionV3/Logits')
    logit_init = tf.variables_initializer(logit_variables)
    
    with tf.name_scope('loss'):
        loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits,labels=y))
    
    with tf.name_scope('logit_optimizer'):
        logit_optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
        logit_train_op = logit_optimizer.minimize(loss,var_list=logit_variables)
    
    aux_variables = slim.get_variables('InceptionV3/AuxLogits')
    mixed_variables = slim.get_variables('InceptionV3/Mixed_7c')
    
    with tf.name_scope('full_optimizer'):
        full_optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.0001)
        full_train_op = full_optimizer.minimize(loss,var_list=[mixed_variables,aux_variables,logit_variables])
    
    with tf.name_scope('accuracy'):
        correct_prediction = tf.equal(tf.argmax(logits,1),tf.argmax(y,1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
    
    
    print("Finalizing the Graph ...... !")
    
    writer.add_graph(graph)

Initializing the graph ..... !
Finalizing the Graph ...... !


In [46]:
!tensorboard --logdir='./graph'

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
2018-05-29 11:15:02.235932: I tensorflow/core/platform/cpu_feature_guard.cc:140] Your CPU supports instructions that this TensorFlow binary was not compiled to use: SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA
2018-05-29 11:15:03.365400: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1356] Found device 0 with properties: 
name: GeForce GTX 1080 Ti major: 6 minor: 1 memoryClockRate(GHz): 1.683
pciBusID: 0000:17:00.0
totalMemory: 10.91GiB freeMemory: 5.09GiB
2018-05-29 11:15:03.365459: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1435] Adding visible gpu devices: 0
2018-05-29 11:15:03.934438: I tensorflow/core/common_runtime/gpu/gpu_device.cc:923] Device interconnect StreamExecutor with st